# Course Bottelneck Index(CBI)

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import re
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
# Loading data
df = pd.read_excel("data/df_grad_filtered.xlsx")


## Demand Ratio(DR)

 Measures how full a course is relative to its cap:
 DR = enrollment / cap
 A value above 1 indicates unmet demand.


### Overall

In [ ]:
# Makeing sure enrollment fields are numeric
df["Total Enroll"] = pd.to_numeric(df["Total Enroll"])
df["Enrollment Capacity"] = pd.to_numeric(df["Enrollment Capacity"])

# Computing Demand Ratio (DR = enrollment ÷ capacity)
df["DR"] = df["Total Enroll"] / df["Enrollment Capacity"]

# Handling divisions by zero or missing capacity
df["DR"] = df["DR"].replace([np.inf, -np.inf], pd.NA)

# Inspecting the result
from IPython.display import display
display(df[[
    "Course Description",
    "Total Enroll",
    "Enrollment Capacity",
    "DR"
]])

,Course Description,Total Enroll,Enrollment Capacity,DR
0,Organization/Information,0,0,NaN
1,Data Warehousing in the Cloud,0,30,0.0
2,Introduction To Archives,0,35,0.0
3,Database Dev And Mgmt,0,35,0.0
4,Intro Info Technology,0,30,0.0
...,...,...,...,...
876,Advanced Game Development,0,28,0.0
877,Applied NLP,36,60,0.6
878,Database Dev And Mgmt,10,35,0.285714
879,Database Dev And Mgmt,4,35,0.114286


### Term By Term

In [ ]:
# Grouping by Term, Course, and Instruction Mode
dr_by_mode = (
    df
    .groupby(
        ["Term", "Course Description", "Instruction Mode"],
        as_index=False
    )
    .agg(
        Total_Enroll   = ("Total Enroll", "sum"),
        Total_Capacity = ("Enrollment Capacity", "sum"),
    )
)

# Computing the Demand Ratio at that level
dr_by_mode["DR"] = dr_by_mode["Total_Enroll"] / dr_by_mode["Total_Capacity"]

# Sorting for easy viewing
dr_by_mode = dr_by_mode.sort_values(
    ["Term", "Course Description", "Instruction Mode"]
)

# Preview
from IPython.display import display
display(dr_by_mode)

,Term,Course Description,Instruction Mode,Total_Enroll,Total_Capacity,DR
0,Fall 2021,Algorithms for Games,In Person,1,10,0.100000
1,Fall 2021,Applied Cyberinfrastruct Conc,In Person,5,78,0.064103
2,Fall 2021,Bayesian Modeling & Inference,Live Online,5,35,0.142857
3,Fall 2021,Cataloging+Metadata Mgmt,Fully Online,11,40,0.275000
4,Fall 2021,Collection Management,In Person,0,30,0.000000
...,...,...,...,...,...,...
375,Spring 2025,STEM Games,Fully Online,4,70,0.057143
376,Spring 2025,Social Justice in Info Service,Fully Online,29,70,0.414286
377,Spring 2025,Special Topics in LIS,Fully Online,0,0,NaN
378,Spring 2025,Virtual Reality,Fully Online,13,65,0.200000


In [ ]:
# Ensuring numeric types for DR calculation
df["Total Enroll"]        = pd.to_numeric(df["Total Enroll"], errors="coerce")
df["Enrollment Capacity"] = pd.to_numeric(df["Enrollment Capacity"], errors="coerce")

# Computing DR at the course‐term level (including Subject Code & Catalog Number)
dr_course_term = (
    df
    .groupby(
        ["Subject Code", "Catalog Number", "Course Description", "Term"],
        as_index=False
    )
    .agg(
        Total_Enroll   = ("Total Enroll",    "sum"),
        Total_Capacity = ("Enrollment Capacity", "sum")
    )
)
dr_course_term["DR"] = dr_course_term["Total_Enroll"] / dr_course_term["Total_Capacity"]

# Pivot rows = Subject Code + Catalog Number + Course, columns = Term, values = DR
dr_over_time = dr_course_term.pivot(
    index=["Subject Code", "Catalog Number", "Course Description"],
    columns="Term",
    values="DR"
).reset_index()

# Sorting the term‐columns chronologically
term_cols = dr_over_time.columns.drop(["Subject Code", "Catalog Number", "Course Description"])
term_order = (
    pd.to_datetime(
        term_cols.to_series()
                 .str.replace(
                     r"^(Spring|Fall) (\d{4})$",
                     lambda m: f"{m.group(2)}-{'01' if m.group(1)=='Spring' else '08'}-01",
                     regex=True
                 )
    )
    .sort_values()
)
dr_over_time = dr_over_time[["Subject Code", "Catalog Number", "Course Description"] + term_order.index.tolist()]

# Saving to Excel
output_path = "data/DR_CBI.xlsx"
dr_over_time.to_excel(output_path, index=False)

from IPython.display import display
display(dr_over_time.head(10))



Term,Subject Code,Catalog Number,Course Description,Fall 2021,Spring 2022,Fall 2022,Spring 2023,Fall 2023,Spring 2024,Fall 2024,Spring 2025
0,BE,529,Applied Cyberinfrastruct Conc,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,0.025000
1,GAME,526,Game AI,NaN,NaN,NaN,NaN,NaN,0.600000,NaN,0.200000
2,INFO,501,Designing an Installation,0.028571,0.0,0.000000,NaN,NaN,NaN,NaN,NaN
3,INFO,502,Data Ethics,NaN,NaN,NaN,NaN,0.375000,0.637500,0.430657,0.512500
4,INFO,505,Foundations of Information,0.371429,NaN,0.342857,NaN,0.419355,NaN,NaN,0.628571
5,INFO,507,Information Research Methods,NaN,0.5,NaN,0.3000,0.379310,NaN,0.275862,NaN
6,INFO,510,Bayesian Modeling & Inference,0.142857,NaN,NaN,NaN,NaN,NaN,0.420000,0.266667
7,INFO,511,Foundations of Data Science,NaN,NaN,NaN,NaN,NaN,NaN,0.585714,0.651852
8,INFO,514,Computational Social Science,NaN,NaN,0.228571,0.3125,0.428571,0.935484,NaN,NaN
9,INFO,515,Organization/Information,0.085714,NaN,0.057971,0.0000,0.013889,0.028571,0.000000,0.030000


## Offering Frequency Score (OFS)

Captures how often a course is offered:
1: Every semester
2: Once a year
3: Less than once a year



In [8]:
import pandas as pd

# 1. Load your datasets
df_faculty = pd.read_excel("data/df_faculty_load_filtered.xlsx")
df_grad    = pd.read_excel("data/df_grad_filtered.xlsx")

# 2. Normalize Catalog Number to string
df_faculty["Catalog Number"] = df_faculty["Catalog Number"].astype(str)
df_grad   ["Catalog Number"] = df_grad   ["Catalog Number"].astype(str)

# 3. Parse Term into Season + Year, then compute academic‐year start
#    (Fall YYYY → academic year starts YYYY; Spring YYYY → starts YYYY–1)
df_faculty[["Season", "Year"]] = df_faculty["Term"].str.split(expand=True)
df_faculty["Year"] = df_faculty["Year"].astype(int)
df_faculty["AcadYearStart"] = df_faculty.apply(
    lambda r: r["Year"] if r["Season"].lower() == "fall" else r["Year"] - 1,
    axis=1
)

# 4. Compute totals for scoring
total_terms      = df_faculty["Term"].nunique()       # e.g. Fall21, Spring22, ...
total_acad_years = df_faculty["AcadYearStart"].nunique()  # e.g. 2021,2022,...

# 5. Group to compute how many terms and academic years each course is offered
ofs_cbi = (
    df_faculty
    .groupby(["Subject Code","Catalog Number"], as_index=False)
    .agg(
        terms_offered      = ("Term",      "nunique"),
        acad_years_offered = ("AcadYearStart","nunique")
    )
)

# 6. Score OFS_CBI using academic years
def score_ofs(row):
    if row["terms_offered"] == total_terms:
        return 1    # offered every single term
    elif row["acad_years_offered"] == total_acad_years:
        return 2    # at least once each academic year
    else:
        return 3    # less than once per academic year

ofs_cbi["OFS_CBI"] = ofs_cbi.apply(score_ofs, axis=1)

# 7. Bring in Course Description
course_map = (
    df_grad[["Subject Code","Catalog Number","Course Description"]]
    .drop_duplicates(subset=["Subject Code","Catalog Number"])
)
ofs_cbi = ofs_cbi.merge(
    course_map,
    on=["Subject Code","Catalog Number"],
    how="left"
)

# 8. Save to Excel
ofs_cbi.to_excel("data/OFS_CBI.xlsx", index=False)

# 9. Preview
from IPython.display import display
display(ofs_cbi.head(10))

,Subject Code,Catalog Number,terms_offered,acad_years_offered,OFS_CBI,Course Description
0,BE,529,2,2,3,Applied Cyberinfrastruct Conc
1,GAME,526,2,2,3,Game AI
2,INFO,501,3,2,3,Designing an Installation
3,INFO,502,4,2,3,Data Ethics
4,INFO,505,7,4,2,Foundations of Information
5,INFO,507,4,4,2,Information Research Methods
6,INFO,510,3,2,3,Bayesian Modeling & Inference
7,INFO,511,2,1,3,Foundations of Data Science
8,INFO,514,5,3,3,Computational Social Science
9,INFO,515,8,4,1,Organization/Information


## Instructor Availability Score (IAS)

 Reflects how many instructors can teach the course:
 IAS = 1 / number of instructors
 Higher scores indicate a staffing bottleneck.


In [11]:
# Counting distinct instructors per course key
instr_counts = (
    df_faculty
    .groupby(["Subject Code", "Catalog Number"], as_index=False)
    .agg(n_instructors=("Psuedonymn", "nunique"))
)

# Computing IAS = 1 / number of instructors
instr_counts["IAS"] = 1 / instr_counts["n_instructors"]

# Pulling in the Course Description via the same key
course_map = (
    df_grad[["Subject Code", "Catalog Number", "Course Description"]]
    .drop_duplicates(subset=["Subject Code", "Catalog Number"])
)
ias_cbi = instr_counts.merge(
    course_map,
    on=["Subject Code", "Catalog Number"],
    how="left"
)

# Reordering columns
ias_cbi = ias_cbi[[
    "Subject Code",
    "Catalog Number",
    "n_instructors",
    "IAS"
]]

# Saveing to Excel
output_path = "data/IAS_CBI.xlsx"
ias_cbi.to_excel(output_path, index=False)
print(f"✔️ Saved Instructor Availability Score to {output_path}")

# Preview
from IPython.display import display
display(ias_cbi.head(10))

✔️ Saved Instructor Availability Score to data/IAS_CBI.xlsx


,Subject Code,Catalog Number,n_instructors,IAS
0,BE,529,5,0.200000
1,GAME,526,1,1.000000
2,INFO,501,2,0.500000
3,INFO,502,6,0.166667
4,INFO,505,10,0.100000
5,INFO,507,2,0.500000
6,INFO,510,4,0.250000
7,INFO,511,3,0.333333
8,INFO,514,2,0.500000
9,INFO,515,8,0.125000


## Prerequisite Complexity Score (PCS)
Quantifies how difficult it is to access a course based on its prerequisites:
a. 0: No prerequisites
b. 1: 1–2 prerequisites
c. 2+: Complex or sequenced prerequisites

In [ ]:
# Loading your prerequisite list
df_pr = pd.read_excel("data/Course List w. Pre-Reqs.xlsx")

# Renaming columns to match your keys
df_pr = df_pr.rename(columns={
    "Subject": "Subject Code",
    "Catalog #": "Catalog Number"
})

# Normalizing 'Catalog Number' to string
df_pr["Catalog Number"] = df_pr["Catalog Number"].astype(str)

# Counting how many prerequisites each course has, with explicit recode for 'Y' and '-'
def count_prereqs(pr_str):
    if pr_str == "Y":
        return 1
    if pr_str == "-":
        return 0
    if pd.isna(pr_str) or not str(pr_str).strip():
        return 0
    parts = re.split(r"[;,/]", str(pr_str))
    return len([p for p in parts if p.strip()])

df_pr["n_prereqs"] = df_pr["Requirements"].apply(count_prereqs)

# Maping counts → PCS
def assign_pcs(n):
    if n == 0:
        return 0
    elif n <= 2:
        return 1
    else:
        return 2

df_pr["PCS"] = df_pr["n_prereqs"].apply(assign_pcs)

# Selecting and reorder the columns
pcs_cbi = df_pr[[
    "Subject Code",
    "Catalog Number",
    "n_prereqs",
    "PCS"
]].drop_duplicates()

# Saving to Excel as PCS_CBI.xlsx
output_path = "data/PCS_CBI.xlsx"
pcs_cbi.to_excel(output_path, index=False)
print(f"✔️ Saved Prerequisite Complexity Score to {output_path}")

# Preview
from IPython.display import display
display(pcs_cbi.head(10))

✔️ Saved Prerequisite Complexity Score to data/PCS_CBI.xlsx


,Subject Code,Catalog Number,n_prereqs,PCS
0,ESOC,150B1,0,0
1,ESOC,199,1,1
2,ESOC,200,0,0
3,ESOC,210,0,0
4,ESOC,211,0,0
5,ESOC,212,0,0
6,ESOC,213,0,0
7,ESOC,214,0,0
8,ESOC,299,1,1
9,ESOC,300,0,0


## Adding Meeting Days and Instruction Mode

In [ ]:
# Loading your raw grad dataset
df = pd.read_excel("data/df_grad_filtered.xlsx")

# Building a schedule‐only DataFrame with exactly the columns you want
schedule = (
    df
    .groupby(
        ["Subject Code", "Catalog Number", "Course Description"],
        as_index=False
    )
    .agg({
        "Meeting Days":       lambda x: "; ".join(sorted(x.dropna().unique())),
        "Instruction Mode":   lambda x: "; ".join(sorted(x.dropna().unique())),
        "Meeting Time Start": lambda x: "; ".join(sorted(x.dropna().astype(str).unique()))
    })
)

# Bucketing start times into a single “Meeting Time” label
def bucket_label(t):
    hh = int(t.split(":",1)[0])
    if hh < 11:
        return "Before 11 am"
    elif hh < 14:
        return "11 am – 2 pm"
    else:
        return "2 pm & after"

def make_time_label(starts):
    labels = {bucket_label(t) for t in starts.split("; ") if t}
    return "; ".join(sorted(labels))

schedule["Meeting Time"] = schedule["Meeting Time Start"].apply(make_time_label)

# Selecting only the final six columns
df_final = schedule[[
    "Subject Code",
    "Catalog Number",
    "Course Description",
    "Meeting Days",
    "Instruction Mode",
    "Meeting Time"
]]

# Saving to Excel
output_path = "data/Details_CBI.xlsx"
df_final.to_excel(output_path, index=False)

from IPython.display import display
display(df_final.head())


,Subject Code,Catalog Number,Course Description,Meeting Days,Instruction Mode,Meeting Time
0,BE,529,Applied Cyberinfrastruct Conc,-; TR,Fully Online; In Person,Before 11 am
1,GAME,526,Game AI,TR,In Person,2 pm & after
2,INFO,501,Designing an Installation,TR,In Person; Live Online,11 am – 2 pm; 2 pm & after
3,INFO,502,Data Ethics,-; TR,Fully Online; In Person,11 am – 2 pm; Before 11 am
4,INFO,505,Foundations of Information,M; W,In Person,2 pm & after
